In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.pmtspec
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import scipy.interpolate
import scipy.special
import scipy.stats
import scipy.signal

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
def midtime(t):
    t = np.array(t)
    return (t[1:] + t[:-1]) *.5
    
class InterpCounts:
    def __init__(self, t, counts, kind='linear'):
        self.t = t
        self.counts = counts
        self.kind = kind
        
        self._interp_func_ = None
    
    def interp(self, t, kind=None):
        if kind is not None and kind != self.kind:
            self.kind = kind
            self._interp_func_ = None  # set it new
            
        if self._interp_func_ is None:
            self._interp_func_ = scipy.interpolate.interp1d(self.t, 
                                                            self.counts, 
                                                            kind=self.kind, 
                                                            assume_sorted=True,
                                                            bounds_error=False)
            
        return self._interp_func_(t)
    
    def rate(self, t, kind=None):
        reads, b, n = scipy.stats.binned_statistic(self.t, None, statistic='count', bins=t)
        rate = np.diff(self.interp(t, kind)) / np.diff(t)
        return np.ma.array(rate, mask=reads==0)
    
    @property
    def rate_0(self,):
        return np.diff(self.counts) / np.diff(self.t)

In [ ]:
def stair_scatter(df, color=None, ax=None, columns=None, **kwargs):
    if columns is None:
        columns = df.columns
    rows = columns[1:] 
    cols = columns[:-1]
    
    # ax[row, col]
    if ax is None:
        fig, ax = plt.subplots(nrows=len(rows), 
                       ncols=len(cols), 
                       sharex='col', sharey='row', 
                       squeeze=False, 
                       **kwargs)
        
        for j, cols_j in enumerate(cols):
            ax[-1,j].set_xscale('log')
            ax[-1,j].set_xlabel(cols_j, rotation=0) #rotation=70
        for i, row_i in enumerate(rows):
            ax[i, 0].set_ylabel(row_i, rotation=90) #rotation=30)
            ax[i, 0].set_yscale('log')
        for i, row_i in enumerate(rows):
            for j, cols_j in enumerate(cols):
                if j<=i:
                    ax[i, j].grid()
                else:
                    ax[i,j].axis('off')

    for i, row_i in enumerate(rows):
        for j, cols_j in enumerate(cols):
            if j<=i:#cols_j != row_i:
    #             ax[i, j].text(0.5, 0.5, f'x:{cols_j}\ny:{row_i}', 
    #                           horizontalalignment='center',
    #                           verticalalignment='center', 
    #                           transform=ax[i, j].transAxes)
                ax[i, j].scatter(df[cols_j], df[row_i], s=1, c=color, alpha=.1)
        
    return ax

# Very basic example

In [ ]:
# readoout delta time and delta counts
dt = np.array([.9, 1.1, 1.2, 1.01, 0.85, 1.11, 0.94, 3.8, 0.81, 1.1])
dc = np.array([ 2,   1,   3,   16,   13,    8,    5,   6,    2,   1])

# counts
c = np.cumsum([0, *dc])

# readoout time, shift the time a bit
t = np.cumsum([0, *dt]) - 0.5

# artificial readoout time with fixed interval
t_a = np.arange(int(t[0]), int(t[-1])+1, 1)

# plot time, more points
t_p = np.linspace(int(t[0]), int(t[-1]), 1000)

In [ ]:
interp_counts = InterpCounts(t, c)

fig, ax = plt.subplots(nrows=2, sharex=True, squeeze=False, figsize=[10,8])
ax = ax.flatten()

ax[0].plot(interp_counts.t, interp_counts.counts, 'o-', label='data')

for i in ['linear', 'quadratic', 'cubic'][:1]:
    lin, = ax[0].plot(t_a, interp_counts.interp(t_a, i), 'x', label=f'interplated {i}')
    ax[0].plot(t_p, interp_counts.interp(t_p, i), ':', color=lin.get_color())
    
ax[0].set_ylabel('Cumulative Counts')
ax[0].grid()
ax[0].legend()

ax[1].plot(midtime(interp_counts.t), 
         np.diff(interp_counts.counts)/np.diff(interp_counts.t), 'o-', ms=7, label='data')

for i in ['linear', 'quadratic', 'cubic'][:1]:
    reads, b, n = scipy.stats.binned_statistic(t, None, statistic='count', bins=t_a)
    rate = interp_counts.rate(t_a, i)

    rate = np.ma.array(rate, mask=reads==0)
    
    lin, = ax[1].plot(midtime(t_a), 
                      rate, 
                    ':x', label=f'interplated {i}')
    lin, = ax[1].plot(midtime(t_a)[rate.mask], 
                      rate.data[rate.mask], 
                      'X', label=f'excluded',
                      color=lin.get_color(), ms=4)
#     ax[1].plot(interp_counts.midtime(t_p), interp_counts.rate(t_p, i), ':', color=lin.get_color())



ax[1].set_ylabel('Rate [Hz]')
ax[1].grid()
ax[1].legend()

ax[-1].set_xlabel('Time')
ax[-1].set_xticks(t_a)
ax[-1].set_xticks(midtime(t_a), minor=True)

plt.tight_layout()

In [ ]:
plt.figure()
plt.plot(midtime(interp_counts.t), 
         np.diff(interp_counts.counts)/np.diff(interp_counts.t), 'o-', label='data')

for i in ['linear', 'quadratic', 'cubic']:
    lin, = plt.plot(midtime(t_a), 
                    interp_counts.rate(t_a, i), 
                    ':x', label=f'interplated {i}')
#     plt.plot(interp_counts.midtime(t_p), interp_counts.rate(t_p, i), ':', color=lin.get_color())

plt.xlabel('Time')
plt.ylabel('Rate [Hz]')
plt.grid()
plt.legend()
plt.tight_layout()

In [ ]:
# freq_min = 1e2
# freq_max = 3e3
# rand = np.random.random_sample(length)
# t_second = np.cumsum([0, *1./freq])

# counts_min = 0
# counts_max = 3e3
# rand = np.random.random_sample(length)

# counts = (counts_max - counts_min) * rand + counts_min
# counts = np.cumsum([0, *counts])

freq_a = 1.
freq_r = 3.

# time in seconds
t_p = np.linspace(-.5, 7.5, 1000000)  # time to plot line
t_a = np.arange(0, t_p[-1], 1./freq_a)
t_r = np.arange(-.5, t_p[-1], 1./freq_r)
t_r += (np.random.random_sample(len(t_r))-.5) * 1./freq_r * .5

In [ ]:
def gamma(x, alpha, beta):
    y = np.zeros_like(x)
    mask = x >= 0
    
    y[mask] = ((beta**alpha) * (x[mask]**(alpha-1)) * (np.exp(-beta*x[mask]))) / scipy.special.gamma(alpha)
    return y

def gamma_real(x, alpha, beta, x0, y0, scale_x, scale_y):
    return gamma((x - x0) / scale_x, alpha, beta) * scale_y + y0

scale_x = 1
scale_y = 1e5

sig_baselinen = 6e2 # Hz
sig_baselinen_min = 1e2
sig_baselinen_max = 2e3

gamma_p0 = (3.3, 4.5, 
            (t_a[-1]-t_a[0])*.2,
            sig_baselinen, 
            scale_x, scale_y)

# '+sig_baselinen' or '+sig_baselinen*(t_p[1]-t_p[0])' for log scale, doesn't influence the rest
sig_p = gamma_real(midtime(t_p), *gamma_p0) + sig_baselinen
counts_p  = np.cumsum([0, *(sig_p * np.diff(t_p))]) + sig_baselinen

# real counter reads, add some error
# counters must increase, therefore add the error to np.diff and calculate the cumsum
# - add an linear error: counts_r
# - add an frequency error: sig_baselinen*np.diff(t_r)
# err_weigth defines how much the single errors account in the end
err_weigth = .2
counts_r = np.interp(t_r, t_p, counts_p)
counts_r = np.diff(counts_r)
counts_r += ((np.random.random_sample(len(t_r)-1)-.5) * (sig_baselinen_max - sig_baselinen_min) + sig_baselinen_min) *np.diff(t_r)
# counts_r += (np.random.random_sample(len(t_r)-1)-.5) * counts_r*.1
counts_r = np.cumsum([0, *counts_r]) + sig_baselinen

counts_r = counts_r.astype(int)
counts_p = counts_p.astype(int)

invalid_interval = (4, 6)
mask = (invalid_interval[0] > t_r) | (invalid_interval[1] < t_r)
t_r = t_r[mask]
counts_r = counts_r[mask]

interp_counts = InterpCounts(t_r, counts_r)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True, squeeze=False, figsize=[10,8])
ax = ax.flatten()
# invert the zorder to draw a line from ax[0] to ax[1]
ax[0].set_zorder(1)
ax[1].set_zorder(0)

bbox_dict = dict(boxstyle="round", fc="1", ec='gray', alpha=.75)

ax[0].plot(t_p, counts_p, label='Signal', color='gray')
ax[0].plot(interp_counts.t, interp_counts.counts, 'o', label='TRB Counts', alpha=1, ms=7)
lin, = ax[0].plot(t_a, interp_counts.interp(t_a, 'linear'), 
                  'D', label=f'Interpolated Counts', ms=7, alpha=.7)
ax[0].set_ylabel('Cumulative Counts')

c = interp_counts.interp(t_a, 'linear')
ax[0].annotate("",
               xy=(t_a[1], c[1]), xycoords='data',
               xytext=(t_a[2], c[1]), textcoords='data',
               arrowprops=dict(arrowstyle="-",
                               color="k",
                               shrinkA=.05, shrinkB=.05,
                               patchA=None, patchB=lin,
                               lw=1.,
#                                connectionstyle="angle,angleA=90,angleB=0,rad=0.0"
                               connectionstyle="bar,fraction=-0.1",
                              ),
               )
ax[0].annotate(r"$\Delta t_i = \frac{1}{f_A}$",
               xy=(midtime(t_a)[1], c[1]), xycoords='data',
               xytext=(0, -15), textcoords='offset points',
               horizontalalignment='center', verticalalignment='top',
               fontsize=13,
               bbox = bbox_dict,
               )

ax[0].annotate("",
               xy=(t_a[2], c[1]), xycoords='data',
               xytext=(t_a[2], c[2]), textcoords='data',
               arrowprops=dict(arrowstyle="-",
                               color="k",
                               shrinkA=.05, shrinkB=.05,
                               patchA=lin, patchB=None,
                               lw=1.,
#                                connectionstyle="angle,angleA=90,angleB=0,rad=0.0"
                               connectionstyle="bar,fraction=-0.05",
                              ),
               )
an1 = ax[0].annotate(r"$\Delta {c}_i$",
                     # np.exp(midtime(np.log(c))[1])) middle in log scale
                     xy=(t_a[2], np.exp(midtime(np.log(c))[1])), xycoords='data',
                     xytext=(15, 0), textcoords='offset points',
                     horizontalalignment='left', verticalalignment='center',
                     fontsize=13,
                     bbox = bbox_dict,
                     )

ax[0].annotate("Interval without \n TRB Counts",
                     # np.exp(midtime(np.log(c))[1])) middle in log scale
                     xy=(midtime(invalid_interval)[0], c.data[4]), xycoords='data',
                     xytext=(midtime(invalid_interval)[0], .75), textcoords=("data", 'axes fraction'),
                     horizontalalignment='center', verticalalignment='center',
                     fontsize=13,
                     bbox = bbox_dict,
#                      arrowprops = dict(arrowstyle="-|>", 
#                                  patchB=lin2,
#                                  connectionstyle="arc3,rad=0.2"
#                                 )
                     )

ax[0].axvspan(*invalid_interval, alpha=0.15, color='gray')
ax[1].axvspan(*invalid_interval, alpha=0.15, color='gray')


ax[1].plot(midtime(t_p), sig_p, label='Signal', color='gray')
ax[1].plot(midtime(interp_counts.t), 
           np.diff(interp_counts.counts)/np.diff(interp_counts.t), 'o', 
           label='TRB Rate', alpha=1, ms=7)

rate = interp_counts.rate(t_a, 'linear')
lin, = ax[1].plot(midtime(t_a), 
                  rate, 
                  'D', label=f'Interpolated Rate', ms=7, alpha=.7)
lin2, = ax[1].plot(midtime(t_a)[rate.mask], 
           rate.data[rate.mask], 
           'D', label=f'Exclude Inter. Rate', ms=7, alpha=.7)

an1 = ax[1].annotate(r"$r_i = \frac{\Delta {c}_i}{\Delta t_i}$",
               # np.exp(midtime(np.log(c))[1])) middle in log scale
               xy=(midtime(t_a)[1], rate[1]), xycoords='data', # ax[1].transData,
               xytext=(30, -30), textcoords='offset points',
#                xytext=(t_a[3], c[1]), textcoords='data',
               horizontalalignment='center', verticalalignment='top',
               fontsize=13,
               bbox = bbox_dict,
               arrowprops = dict(arrowstyle="-|>", 
                                 patchB=lin,
                                 connectionstyle="arc3,rad=0.2"
                                )
               )

an1 = ax[1].annotate("Interval without \n TRB Counts",
                     # np.exp(midtime(np.log(c))[1])) middle in log scale
                     xy=(midtime(invalid_interval)[0], c.data[4]), xycoords='data',
                     xytext=(midtime(invalid_interval)[0], .75), textcoords=("data", 'axes fraction'),
                     horizontalalignment='center', verticalalignment='center',
                     fontsize=13,
                     bbox = bbox_dict,
#                      arrowprops = dict(arrowstyle="-|>", 
#                                  patchB=lin2,
#                                  connectionstyle="arc3,rad=0.2"
#                                 )
                     )

ax[1].annotate("Exclude rates",
                     # np.exp(midtime(np.log(c))[1])) middle in log scale
                     xy=(.5,.5), xycoords=an1,
                     xytext=(0, -75), textcoords='offset points',
                     horizontalalignment='center', verticalalignment='top',
                     fontsize=13,
                     bbox = bbox_dict,
                     arrowprops = dict(arrowstyle="<|-", 
                                 patchB=an1.get_bbox_patch(),
#                                  connectionstyle="arc3,rad=0.2"
                                )
                     )

# an1.draggable()
    
ax[1].set_ylabel('Rate [Hz]')
ax[-1].set_xlabel('Time [a.u.]')

for ax_i in ax:
    ax_i.grid()
    ax_i.legend(loc='upper left')
    ax_i.set_yscale('log')
    
# ax[0].grid(which='major', axis='both', linestyle='-')
ax[1].grid(which='minor', axis='x', linestyle=':')
    
ax[-1].set_xticks(t_a)
ax[-1].set_xticks(midtime(t_a), minor=True)
ax[-1].set_xlim((t_p.min(), t_p.max()))
plt.tight_layout()

$r_{raw, i} = \frac{\Delta c_{raw, i}}{\Delta t_{raw, i}}; r_{i} = \frac{\Delta c_{i}}{\Delta t_{i}}; \Delta t_A = \frac{1}{\Delta f_A}$

In [ ]:
# freq_min = 1e2
# freq_max = 3e3
# rand = np.random.random_sample(length)
# t_second = np.cumsum([0, *1./freq])

# counts_min = 0
# counts_max = 3e3
# rand = np.random.random_sample(length)

# counts = (counts_max - counts_min) * rand + counts_min
# counts = np.cumsum([0, *counts])

def gamma(x, alpha, beta):
    y = np.zeros_like(x)
    mask = x >= 0
    
    y[mask] = ((beta**alpha) * (x[mask]**(alpha-1)) * (np.exp(-beta*x[mask]))) / scipy.special.gamma(alpha)
    return y

def gamma_real(x, alpha, beta, x0, y0, scale_x, scale_y):
    return gamma((x - x0) / scale_x, alpha, beta) * scale_y + y0

freq_a = 1e1
freq_r = 1e3

# time in seconds
t_p = np.linspace(-.5, 100.5, 1000000)  # time to plot line
t_a = np.arange(  0, t_p[-1], 1./freq_a)
t_r = np.arange(-.5, t_p[-1], 1./freq_r)
t_r += (np.random.random_sample(len(t_r))-.5) * 1./freq_r * .25

scale_x = 1
scale_y = 1e5

sig_baselinen = 6e2 # Hz
sig_baselinen_min = 1e2
sig_baselinen_max = 1.1e3

gamma_p0 = (3.3, 4.5, 
            (t_a[-1]-t_a[0])*.2,
            sig_baselinen, 
            scale_x, scale_y)

# '+sig_baselinen' or '+sig_baselinen*(t_p[1]-t_p[0])' for log scale, doesn't influence the rest
sig_p = gamma_real(midtime(t_p), *gamma_p0) + sig_baselinen
counts_p  = np.cumsum([0, *(sig_p * np.diff(t_p))]) + sig_baselinen

# real counter reads, add some error
# counters must increase, therefore add the error to np.diff and calculate the cumsum
# - add an linear error: counts_r
# - add an frequency error: sig_baselinen*np.diff(t_r)
# err_weigth defines how much the single errors account in the end
err_weigth = .2
counts_r = np.interp(t_r, t_p, counts_p)
counts_r = np.diff(counts_r)
# counts_r += ((np.random.random_sample(len(t_r)-1)-.5) * (sig_baselinen_max - sig_baselinen_min) + sig_baselinen_min) * np.diff(t_r)
# counts_r += (np.random.random_sample(len(t_r)-1)-.5) * counts_r*.1
counts_r = np.cumsum([0, *counts_r]) # + sig_baselinen

counts_r = counts_r.astype(int)
counts_p = counts_p.astype(int)

interp_counts = InterpCounts(t_r, counts_r)

In [ ]:
plt.figure()
plt.hist([np.diff(t_rr)], bins=1000)
plt.grid()

plt.figure()
plt.hist([np.diff(t_r)], bins=1000)
plt.grid()

In [ ]:
t_p = np.linspace(-.5, 10.5, 10)  # time to plot line

# t_r = np.arange(-.5, t_p[-1], 1./freq_r)
t_r = np.linspace(-.5, t_p[-1], int(freq_r * (t_p[-1]+.5)))
t_rr = t_r + (np.random.random_sample(len(t_r))-.5) * 1./freq_r * .25

plt.figure()

plt.plot(np.diff(t_r), 'o')
plt.plot(np.diff(t_rr), 'o', alpha=.1)
plt.grid()

In [ ]:
fig, ax = plt.subplots(nrows=2, 
                       sharex=True, 
                       squeeze=False, 
                       figsize=[10,8])
ax = ax.flatten()

ax[0].plot(t_p, counts_p, color='gray', label='Signal')
ax[0].scatter(interp_counts.t, interp_counts.counts, #'o', 
              label='Data', alpha=1, s=1)

for i in ['linear', 'quadratic', 'cubic']:
    lin, = ax[0].plot(t_a, interp_counts.interp(t_a, i), ':', label=f'Interplat {i}')
#     plt.plot(t_p, interp_counts.interp(t_p, i), ':', color=lin.get_color())
    
ax[0].set_ylabel('Cumulative Counts')


ax[1].plot(midtime(t_p), sig_p, color='gray', label='Signal')
ax[1].scatter(midtime(interp_counts.t), interp_counts.rate_0, #'o', 
              label='data', alpha=.5, s=1
             )

for i in ['linear', 'quadratic', 'cubic']:
    lin, = plt.plot(midtime(t_a), 
                    interp_counts.rate(t_a, i), 
                    ':', label=f'Interplat {i}', ms=5, alpha=1)
#     plt.plot(interp_counts.midtime(t_p), interp_counts.rate(t_p, i), ':', color=lin.get_color())

ax[1].set_ylabel('Rate [Hz]')
ax[-1].set_xlabel('Time')

for ax_i in ax:
    ax_i.grid()
    ax_i.legend()
    ax_i.set_yscale('log')
    
plt.tight_layout()

In [ ]:
plt.figure()
t_r = np.arange(-.5, t_p[-1], 1./freq_r)

hist, bin_edges = np.histogram(1./np.diff(t_r), bins=np.linspace(0,1e4,int(1e4)), density=True)
plt.plot(bin_edges, [0, *hist], label='Data')

# t_r += (np.random.random_sample(len(t_r))-.5) * 1./freq_r * .25
t_r += 1./np.random.normal(loc=1e3, scale=2e2, size=len(t_r))
hist, bin_edges = np.histogram(1./np.diff(t_r), bins=np.linspace(0,1e4,int(1e4)), density=True)

plt.plot(bin_edges, [0, *hist], label='Data')
plt.yscale('log')

In [ ]:
plt.figure()
counts_r = np.interp(t_r, t_p, counts_p)
hist, bin_edges = np.histogram(np.diff(counts_r) / np.diff(t_r), bins=100)
plt.plot(bin_edges, [0, *hist], label='Data')
plt.yscale('log')

In [ ]:
plt.figure()

rate = interp_counts.rate_0
rate_min, rate_max = np.trunc(rate.min()), np.ceil(rate.max())
bin_edges = np.linspace(rate_min, rate_max, int(rate_max-rate_min+1))
hist, bin_edges = np.histogram(rate, bin_edges, density=True)
plt.plot(bin_edges, [0, *hist], label='Data')
    
for i in ['linear', 'quadratic', 'cubic'][:1]:
    rate = interp_counts.rate(t_a, i)
    rate_min, rate_max = np.trunc(rate.min()), np.ceil(rate.max())
    bin_edges = np.linspace(rate_min, rate_max, int(rate_max-rate_min+1))
    hist, bin_edges = np.histogram(rate, bin_edges, density=True)
    
    plt.plot(bin_edges, [0, *hist], label=i)
    
plt.plot()
plt.grid(axis='y')
plt.grid(axis='x', which='both')
# arg_max = hist.argmax()
# plt.xlim(rate[arg_max]-100, rate[arg_max]+100)
plt.xlim(0, 4e3)
plt.yscale('log')
plt.xlabel('Rate [Hz]')
plt.ylabel('Probability')
plt.legend()
plt.tight_layout()

In [ ]:
interp_counts.rate_0.shape

# Signal generator

In [ ]:
def random(size, x_min=0, x_max=1):
    return np.random.random_sample(int(size)) * (x_max-x_min) + x_min

def random_beta(size, x_min=0, x_max=1, alpha=2, beta=5):
    return np.random.beta(alpha, beta, int(size)) * (x_max-x_min) + x_min

t_freq = 1.
t_max = 3600. * 24. * 7.
t = np.arange(0, t_max, 1./t_freq)
peak_freq = 1./240. # every minute
peak_n = int(t_max * peak_freq)
baseline = 2e3
              
df_random = pd.DataFrame(data={'x0': random(peak_n, 0, t_max),
                               'alpha': random(peak_n, 0.5, 5.),
                               'beta': random(peak_n, 0.5, 5.),
                               'scale_y': random_beta(peak_n, baseline, 1e6),
                               'scale_x': random(peak_n, 2., 20.),
                               'y0': 0,
                              }
                        )
              
df_random.sort_values('x0', inplace=True)

sig = np.ones_like(t) * baseline
for ind, values in df_random.iterrows():
    sig += gamma_real(t, **values.to_dict())

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t, sig, '-', ms=1, alpha=1)
# for ind, values in df_random.iterrows():
#     y  = np.ma.masked_less(gamma_real(t, **values.to_dict()), baseline)
#     plt.plot(t, y, color='gray', alpha=.5)
    
plt.yscale('log')
plt.grid()

In [ ]:
stair_scatter(df_random, columns=['x0','alpha' ,'beta','scale_y','scale_x',], figsize=(8,8))
plt.tight_layout()

In [ ]:
peaks, properties =  scipy.signal.find_peaks(sig,
                               height=(None, None),
                               threshold=(None, None),
                               prominence=(None, None),
                               width=(None, None),
                               plateau_size=(None, None),
                              )
"""
* 'peak_heights':  the height of each peak in `x`.
* 'left_thresholds', 'right_thresholds'
      peaks vertical distance to its neighbouring samples.
* 'prominences', 'right_bases', 'left_bases'
      See `peak_prominences` for a description of their content.
* 'width_heights', 'left_ips', 'right_ips'
      See `peak_widths` for a description of their content.
* 'plateau_sizes', 'left_edges', 'right_edges'
      indices of a peak's edges (edges are still part of the plateau) and the calculated plateau sizes.
      
widths = right_ips - left_ips
"""
properties['peaks'] = peaks
df_peaks = pd.DataFrame(data=properties)
df_peaks['widths_bases'] = df_peaks.right_bases - df_peaks.left_bases
df_peaks['dx_peaks'] = np.diff([0, *df_peaks['peaks']])

peaks_log, properties_log =  scipy.signal.find_peaks(np.log(sig),
                               height=(None, None),
                               threshold=(None, None),
                               prominence=(None, None),
                               width=(None, None),
                               plateau_size=(None, None),
                              )
properties_log['peaks'] = peaks_log
df_peaks_log = pd.DataFrame(data=properties_log)
df_peaks_log['widths_bases'] = df_peaks_log.right_bases - df_peaks_log.left_bases
df_peaks_log['dx_peaks'] = np.diff([0, *df_peaks_log['peaks']])

In [ ]:
columns = [#'plateau_sizes', 'left_thresholds', 'right_thresholds'
           'peak_heights', 'prominences', 'widths_bases', 'widths', 'width_heights', 'dx_peaks']

ax = stair_scatter(df_peaks, ax=None, columns=columns, figsize=(9,9))
plt.tight_layout()

ax = stair_scatter(df_peaks_log, ax=None, columns=columns, figsize=(9,9))
plt.tight_layout()